In [2]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torchvision.models as models
import torchvision.transforms as transforms

In [4]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def train(net, trainLoader, testLoader, optimizer, criterion, epochs):
    net.train()
    testAccuracy = 0
    bestModel = net
    for i in range(epochs):
        totalLoss = 0
        accuracy = 0
        count = 0
        for x, label in trainLoader:
            x = x.to(device)
            label = label.to(device, dtype=torch.long)
            optimizer.zero_grad()
            output = net(x)
            loss = criterion(output, label)
            _, predicted = torch.max(output.data, 1)
            count += len(x)
            accuracy += (predicted == label).sum().item()      
            totalLoss += loss.item()*len(label)
            loss.backward()
            optimizer.step()
        print("Train Loss: {}".format(totalLoss / count))
        print("Train Accuracy: {}".format(accuracy / count))
        if (i % 10 == 0):
            tmpAccuracy = test(net, testLoader, criterion)
            if (tmpAccuracy > testAccuracy):
                testAccuracy = tmpAccuracy
                bestModel = net
                epoch = i
    torch.save(bestModel, "model/epoch"+str(epoch)+"_"+str(testAccuracy)+".pth")
    return net

In [4]:
def test(net, testLoader, criterion):
    net.eval()
    totalLoss = 0
    accuracy = 0
    count = 0
    for x, label in testLoader:
        x = x.to(device)
        label = label.to(device, dtype=torch.long)
        output = net(x)
        loss = criterion(output, label)
        _, predicted = torch.max(output.data, 1)
        count += len(x)
        accuracy += (predicted == label).sum().item()
        totalLoss += loss.item()*len(label)
    print("Test Loss: {}".format(totalLoss / count))
    print("Test Accuracy: {}".format(accuracy / count))
    return (accuracy / count)

In [5]:
class VGG16_model(nn.Module):
    def __init__(self, numClasses=7):
        super(VGG16_model, self).__init__()
        self.vgg16 = nn.Sequential(
            nn.Conv2d(3,64,3),
            nn.ReLU(inplace=True),
            nn.Conv2d(64,64,3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64,128,3),
            nn.ReLU(inplace=True),
            nn.Conv2d(128,128,3),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(128,256,3),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256,3),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256,3),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,512,3),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,3),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,3),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,3),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,3),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, numClasses)
        )

    def forward(self, x):
        x = self.vgg16(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [6]:
class image_dataset(Dataset):
    def __init__(self, csvFile, rootPath, transform):
        df = pd.read_csv(csvFile)
        self.rootPath = rootPath
        self.xTrain = df['path']
        self.yTrain = pd.factorize(df['label'], sort=True)[0]
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.rootPath, self.xTrain[index]))
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, self.yTrain[index]

    def __len__(self):
        return len(self.xTrain.index)

In [7]:
def set_parameter_requires_grad(model, featureExtracting):
    if featureExtracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
def VGG16_pretrained_model(numClasses, featureExtract=True, usePretrained=True):
    model = models.vgg16(pretrained=True)
    set_parameter_requires_grad(model, featureExtract)
    numFtrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(numFtrs,numClasses)
    return model

In [9]:
def main():
    inputSize = 224
    dataTransformsTrain = transforms.Compose([
     transforms.RandomResizedCrop(inputSize),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    dataTransformsTest = transforms.Compose([
     transforms.Resize(inputSize),
     transforms.CenterCrop(inputSize),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    trainDatasets = image_dataset("dataset/train.csv", "dataset/cosmetics-all", dataTransformsTrain)
    testDatasets = image_dataset("dataset/test.csv", "dataset/cosmetics-all", dataTransformsTest)
    dataloadersTrain = torch.utils.data.DataLoader(trainDatasets, batch_size=32, shuffle=True, num_workers=4)
    dataloadersTest = torch.utils.data.DataLoader(testDatasets, batch_size=32, shuffle=False)
    model = VGG16_model(numClasses=7).to(device)
    # model = VGG16_pretrained_model(numClasses=7, featureExtract=True, usePretrained=True).to(device)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss().to(device)
    model_ft = train(model, dataloadersTrain, dataloadersTest, optimizer, criterion, 11)

In [10]:
main()

Train Loss: 1.9372227478496085
Train Accuracy: 0.26541554959785524
Test Loss: 1.9283317277129268
Test Accuracy: 0.30633802816901406
Train Loss: 1.9190211803173784
Train Accuracy: 0.30741733690795353
Train Loss: 1.9013489871498088
Train Accuracy: 0.30741733690795353
Train Loss: 1.886734832494359
Train Accuracy: 0.30741733690795353
Train Loss: 1.8735990189781564
Train Accuracy: 0.30741733690795353
Train Loss: 1.8626856161506182
Train Accuracy: 0.30741733690795353
Train Loss: 1.852935223519749
Train Accuracy: 0.30741733690795353
Train Loss: 1.8446730331611805
Train Accuracy: 0.30741733690795353
Train Loss: 1.8376814830079475
Train Accuracy: 0.30741733690795353
Train Loss: 1.831812980986792
Train Accuracy: 0.30741733690795353
Train Loss: 1.8265583002748396
Train Accuracy: 0.30741733690795353
Test Loss: 1.827178286834502
Test Accuracy: 0.30633802816901406


/home/pj/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type VGG16_model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
